# Parametric Gates via ARP/πₐ Modulation
This notebook estimates iSWAP and CZ gate rates from a parametric drive with phase-mod index β.
We use small-PM Bessel dressing: $J_1(β)$ sets the effective coupling.
These are **first-order**, control-facing numbers for planning; not a full Hamiltonian sim.

In [ ]:
import numpy as np
from mpmath import besselj
import matplotlib.pyplot as plt
g0 = 50e6*2*np.pi  # rad/s
T1=30e-6; Tphi=20e-6


## iSWAP: $J_{\rm eff} \approx g_0 J_1(β)$

In [ ]:
def J_eff_iswap(beta,g0):
    from mpmath import besselj
    return float(g0*besselj(1,beta))
def gate_time_iswap(beta,g0):
    J=J_eff_iswap(beta,g0)
    return np.pi/(2*J) if J>0 else np.inf
def decoh(beta,g0,T1,Tphi):
    t=gate_time_iswap(beta,g0)
    return float(np.exp(-t/T1)*np.exp(-t/Tphi)) if np.isfinite(t) else 0.0
betas=np.linspace(0.01,0.35,80)
tpis=[gate_time_iswap(b,g0) for b in betas]
fids=[decoh(b,g0,T1,Tphi) for b in betas]
fig,ax=plt.subplots(1,2,figsize=(10,4))
ax[0].plot(betas, np.array(tpis)*1e9); ax[0].set_xlabel('β'); ax[0].set_ylabel('tπ (ns)'); ax[0].set_title('iSWAP tπ vs β')
ax[1].plot(betas, fids); ax[1].set_xlabel('β'); ax[1].set_ylabel('F_decoh'); ax[1].set_title('Decoherence bound vs β'); plt.show()

## Detuning sensitivity

In [ ]:
beta=0.15
J=J_eff_iswap(beta,g0)
Deltas=np.linspace(-20e6*2*np.pi,20e6*2*np.pi,201)
tpi=gate_time_iswap(beta,g0)
Ps=[]
for d in Deltas:
    Om=np.sqrt(J**2+d**2); Ps.append((J**2/(J**2+d**2))*np.sin(Om*tpi)**2)
plt.figure(figsize=(6.8,4.2)); plt.plot(Deltas/(2*np.pi*1e6), Ps)
plt.xlabel('Δ (MHz)'); plt.ylabel('Swap probability at tπ'); plt.title('iSWAP detuning sensitivity (β=0.15)'); plt.show()

## CZ proxy via frequency modulation

In [ ]:
kappa=5e6*2*np.pi
rates=[kappa*besselj(1,b) for b in betas]
tpi=[(np.pi/r) if r>0 else np.inf for r in rates]
plt.figure(figsize=(6.8,4.2)); plt.plot(betas, np.array(tpi)*1e9)
plt.xlabel('β'); plt.ylabel('tπ (ns)'); plt.title('CZ time vs β (proxy)'); plt.show()

**Notes:**
- Use your measured β from sidebands to set gate rate: no free parameters.
- Ramsey/echo removes the small carrier shift from the redshift channel before running gates.
- For accurate fidelities, extend this with a full driven JC Hamiltonian + Lindblad.